## Kaggle Submission File on Test Data

In [7]:
import torch
from torchvision.transforms import v2
import pandas as pd
import torch
import os
from PIL import Image
from pathlib import Path
import numpy as np




In [8]:
dino = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')#freeze parameters
for p in dino.parameters():
    p.requires_grad = False

import os
print(os.getcwd())


Using cache found in C:\Users\julia/.cache\torch\hub\facebookresearch_dinov2_main


c:\Users\julia\Documents\Image2Biomass_BME338\scripts


In [9]:
def preprocess(img_folder_rel_path,resize):
    transformer = v2.Compose([
        v2.Resize(size=resize),
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean = [0.485, 0.456, 0.406],
                       std=[0.229, 0.224, 0.225])
        ])
    processed_images = []
    image_paths = []
    folder_path = os.path.join(img_folder_rel_path)
    folder_path = folder_path.replace('\\', '/')
    for file in sorted(os.listdir(folder_path)):
        if file.lower().endswith('jpg'):
            img_path = os.path.join(folder_path,file)
            img_path = img_path.replace('\\', '/')
            img = Image.open(img_path).convert('RGB')
            trans_img = transformer(img)
            processed_images.append(trans_img)
            image_paths.append(img_path)
    print('Images processed')
    print(len(processed_images))
    return processed_images, image_paths

processed_images, image_paths = preprocess("../data/test_2",224)
img_batch = torch.stack(processed_images)

Images processed
1


In [10]:
img_batch = torch.stack(processed_images)
with torch.no_grad():  
    outputs = dino(img_batch)
print(outputs.shape)


torch.Size([1, 384])


In [11]:
features = outputs.detach().cpu().numpy()
derived_dir = Path('../data/derived')
derived_dir.mkdir(parents=True, exist_ok=True)
embeddings_df = pd.DataFrame(features)
embeddings_df.insert(0, 'image_path', image_paths)
embeddings_path = derived_dir / 'dino_subm_embeddings.parquet'
embeddings_df.to_parquet(embeddings_path, index=False)
print(f'Saved embeddings to {embeddings_path} with shape {embeddings_df.shape}')


Saved embeddings to ..\data\derived\dino_subm_embeddings.parquet with shape (1, 385)


In [12]:
embed = pd.read_parquet("../data/derived/dino_subm_embeddings.parquet")
embed["img_id"] = embed["image_path"].str.split("/").str[-1].str.split(".").str[0] # select image id's
print(embed["img_id"])
feature_cols = embed.select_dtypes(include=[np.number]).columns
X = embed[feature_cols].to_numpy(dtype=np.float32)
X_t = torch.tensor(X, dtype=torch.float32)

0    ID4464212
Name: img_id, dtype: object


In [13]:
import torch
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, input_dim=384, hidden_dim=256, output_dim=1, dropout=0.0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.net(x)


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_dim=X_t.shape[1], hidden_dim=256, output_dim=5, dropout=0.2)
state = torch.load("model_1765703764_918", map_location=device)
model.load_state_dict(state)
model.eval()




MLP(
  (net): Sequential(
    (0): Linear(in_features=384, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=5, bias=True)
  )
)

In [15]:
X_t = X_t.to(device)

with torch.no_grad():
    preds = model(X_t).detach().cpu().numpy()

In [ ]:
target_cols = ["Dry_Clover_g", "Dry_Dead_g", "Dry_Green_g", "Dry_Total_g", "GDM_g"]

pred_df = pd.DataFrame(
    preds,
    columns=target_cols
)
pred_df["img_id"] = embed["img_id"].values

submission = pred_df.melt(
    id_vars="img_id",
    value_vars=target_cols,
    var_name="target_name",
    value_name="target"
)

submission["sample_id"] = submission["img_id"] + "_" + submission["target_name"]
submission = submission[["sample_id", "target"]]
print(submission) 
submission.to_csv("../data/submission/submission.csv",index=False)



                sample_id      target
0  ID4464212_Dry_Clover_g   21.615330
1    ID4464212_Dry_Dead_g   33.120743
2   ID4464212_Dry_Green_g   57.027294
3   ID4464212_Dry_Total_g  116.621193
4         ID4464212_GDM_g   82.669510
